In [1]:
#we use sonar dataset
#binary classification prob to differentiate rocks and metals
# M is mine R is rock

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

seed=7
numpy.random.seed(seed)

#load dataset
dataframe=pandas.read_csv("sonar.csv", header=None)
dataset=dataframe.values

inputVar=dataset[:,0:60].astype(float)
outputVar=dataset[:,60]

#since output var is string, we need to convert it to integer values
encoder=LabelEncoder()
encoder.fit(outputVar)
encoded_outputVar=encoder.transform(outputVar)



Using TensorFlow backend.


In [2]:
#model info :
#use stratified k fold cross validation
#single fully connected hidden layer , same number of neurons as input variables
#weights init using small Gaussian random number
#rectifier avtivation function used
#output layer has sigmoid activation function
#logarithmic loss function binary_crossentropy
#Adam optimisation algo for gradient descent


def create_baseline():
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [7]:
# evaluation of the model
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, inputVar, encoded_outputVar, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 1s - loss: 0.6876 - acc: 0.5323      

In [3]:
# standardise the dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, inputVar, encoded_outputVar, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 0s - loss: 0.6483 - acc: 0.6613      
Epoch 2/100
186/186 [==============================] - 0s - loss: 0.5499 - acc: 0.7957     

In [5]:
# smaller model
def create_smaller():
# create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
# Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, inputVar, encoded_outputVar, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 82.68% (3.18%)


In [6]:
# larger model
def create_larger():
# create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
# Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, inputVar, encoded_outputVar, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 84.57% (4.28%)
